In [1]:
from __future__ import annotations

from collections import OrderedDict
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, List, Tuple

import io
import tempfile
import threading

from contextlib import redirect_stdout

import ipywidgets as widgets
from IPython.display import Image, Markdown, clear_output, display

import compare_solvers as cs
from xprof import server as xprof_server


In [2]:
@dataclass(frozen=True)
class SolverConfig:
    key: str
    name: str
    solver_path: Path
    parfile_path: Path

    def read_parfile(self) -> str:
        return self.parfile_path.read_text()


def discover_solvers(base_dir: Path = Path("Solvers")) -> List[SolverConfig]:
    base_dir = base_dir.resolve()
    configs: List[SolverConfig] = []
    if not base_dir.exists():
        raise FileNotFoundError(f"Solver directory not found: {base_dir}")
    for solver_dir in sorted(base_dir.iterdir()):
        if not solver_dir.is_dir():
            continue
        solver_script = None
        for candidate in sorted(solver_dir.glob("solver*.py")):
            solver_script = candidate
            break
        if solver_script is None:
            continue
        parfile = solver_dir / "parfile"
        if not parfile.exists():
            par_candidates = sorted(solver_dir.glob("*.par"))
            if par_candidates:
                parfile = par_candidates[0]
        if not parfile.exists():
            continue
        key = solver_dir.name
        configs.append(
            SolverConfig(
                key=key,
                name=solver_dir.name.replace("_", " "),
                solver_path=solver_script.resolve(),
                parfile_path=parfile.resolve(),
            )
        )
    if len(configs) < 2:
        raise RuntimeError("At least two solver directories with parfiles are required.")
    return configs


def ordered_subset(selected: Iterable[str], ordering: Iterable[str]) -> List[str]:
    selected_set = set(selected)
    return [item for item in ordering if item in selected_set]


In [3]:
def run_comparison(
    solver_inputs: List[Tuple[SolverConfig, str]],
    *,
    reference_index: int,
    refine_factor: int,
    convergence_tol: float,
    max_refinements: int,
) -> Dict[str, object]:
    if len(solver_inputs) < 2:
        raise ValueError("Select at least two solvers.")
    if reference_index < 0 or reference_index >= len(solver_inputs):
        raise ValueError("Reference index is out of range.")
    if max_refinements < 0:
        raise ValueError("max_refinements must be non-negative.")
    if max_refinements > 0 and refine_factor < 2:
        raise ValueError("refine_factor must be at least 2 when requesting refinements.")

    tempdir = tempfile.TemporaryDirectory(prefix="nb_parfiles_")
    tmpdir_path = Path(tempdir.name)
    configs = [cfg for cfg, _ in solver_inputs]
    parfile_texts = [text for _, text in solver_inputs]
    parfile_paths: List[Path] = []
    base_params: List[OrderedDict[str, object]] = []
    for idx, text in enumerate(parfile_texts):
        parfile_path = tmpdir_path / f"solver_{idx}.par"
        parfile_path.write_text(text.rstrip() + "")
        parfile_paths.append(parfile_path)
        base_params.append(cs.read_parfile(parfile_path))

    log_buffer = io.StringIO()
    try:
        with redirect_stdout(log_buffer):
            results: List[cs.SolverResult] = []
            for idx, (config, parfile_path) in enumerate(zip(configs, parfile_paths)):
                name = chr(ord("A") + idx)
                print(f"[Info] Running Solver {name}: {config.solver_path}")
                result = cs.run_solver(name, config.solver_path, parfile_path, refinement_level=0)
                results.append(result)

            if not results:
                raise ValueError("No solver results were produced.")

            reference_config = configs[reference_index]
            reference_params = base_params[reference_index]
            reference_result = results[reference_index]
            previous_result = reference_result
            reference_refinement_level = 0
            convergence_reached = False
            last_difference = 0.0
            last_linf = 0.0

            for level in range(1, max_refinements + 1):
                params_level = cs.refined_parameters(reference_params, level=level, factor=refine_factor)
                with tempfile.TemporaryDirectory(prefix="nb_refine_") as refine_dir:
                    refined_parfile = Path(refine_dir) / "parfile"
                    cs.write_parfile(refined_parfile, params_level)
                    refined_result = cs.run_solver(
                        reference_result.name,
                        reference_config.solver_path,
                        refined_parfile,
                        refinement_level=level,
                    )

                shared_vars = tuple(
                    sorted(
                        set(previous_result.data_history.keys())
                        & set(refined_result.data_history.keys())
                    )
                )
                if not shared_vars:
                    raise ValueError(
                        f"No common variables between refinement levels for solver {reference_result.name}"
                    )

                ratio = cs.grid_refinement_ratio(refined_result.grid.size, previous_result.grid.size)
                l2_diff, linf_diff = cs.compare_histories_between_levels(
                    previous_result,
                    refined_result,
                    ratio=ratio,
                    variables=shared_vars,
                )
                aggregate = cs.aggregate_error(l2_diff)
                linf_aggregate = cs.max_error(linf_diff)
                last_difference = float(aggregate[-1])
                last_linf = float(linf_aggregate[-1])
                print(
                    f"[Info] Solver {reference_result.name} refinement level {level}: "
                    f"aggregate L2 diff = {last_difference:.3e}, "
                    f"aggregate Linf diff = {last_linf:.3e}"
                )

                reference_result = refined_result
                reference_refinement_level = level
                if last_difference <= convergence_tol:
                    convergence_reached = True
                    break

                previous_result = refined_result

            reference_result.parfile = reference_config.parfile_path

            variable_sets = [set(result.data_history.keys()) for result in results if result.data_history]
            variable_sets.append(set(reference_result.data_history.keys()))
            if not variable_sets:
                raise ValueError("No variables available for convergence comparison across solvers")

            common_vars = tuple(sorted(set.intersection(*variable_sets)))
            if not common_vars:
                raise ValueError("No common variables across solvers for convergence comparison")

            per_var_l2 = {var: {} for var in common_vars}
            per_var_linf = {var: {} for var in common_vars}
            combined_l2: Dict[str, object] = {}
            combined_linf: Dict[str, object] = {}
            grid_ratios: Dict[str, int] = {}

            for result in results:
                ratio = cs.grid_refinement_ratio(reference_result.grid.size, result.grid.size)
                grid_ratios[result.name] = ratio
                l2_hist, linf_hist = cs.compare_histories_between_levels(
                    result,
                    reference_result,
                    ratio=ratio,
                    variables=common_vars,
                )
                result.l2_history = l2_hist
                result.linf_history = linf_hist
                if l2_hist:
                    first_key = next(iter(l2_hist))
                    aligned_length = l2_hist[first_key].shape[0]
                    result.times = result.times[:aligned_length]
                combined_l2[result.name] = cs.aggregate_error(result.l2_history)
                combined_linf[result.name] = cs.max_error(result.linf_history)
                for var in common_vars:
                    per_var_l2[var][result.name] = result.l2_history[var]
                    per_var_linf[var][result.name] = result.linf_history[var]

            phi_l2_plot = phi_linf_plot = None
            if "Phi" in per_var_l2:
                phi_l2_plot = cs.plot_error_history(
                    results,
                    per_var_l2["Phi"],
                    ylabel="L2 Error vs Reference [Phi]",
                    filename="solver_l2_errors_phi.png",
                )
                phi_linf_plot = cs.plot_error_history(
                    results,
                    per_var_linf["Phi"],
                    ylabel="Linf Error vs Reference [Phi]",
                    filename="solver_linf_errors_phi.png",
                )

            pi_l2_plot = pi_linf_plot = None
            if "Pi" in per_var_l2:
                pi_l2_plot = cs.plot_error_history(
                    results,
                    per_var_l2["Pi"],
                    ylabel="L2 Error vs Reference [Pi]",
                    filename="solver_l2_errors_pi.png",
                )
                pi_linf_plot = cs.plot_error_history(
                    results,
                    per_var_linf["Pi"],
                    ylabel="Linf Error vs Reference [Pi]",
                    filename="solver_linf_errors_pi.png",
                )

            for result in results:
                print(f"Solver {result.name}: {result.solver_path}")
                print("  Notebook parfile override applied.")
                print(f"  Runtime: {result.runtime:.3f} s")
                print(f"  Final time: {result.final_time:.6e}")
                print(f"  Grid refinement ratio vs reference: {grid_ratios[result.name]:d}x")
                print(f"  Variables compared: {', '.join(common_vars)}")
                final_l2 = combined_l2[result.name][-1]
                final_linf = combined_linf[result.name][-1]
                print(f"  Final combined L2 error: {final_l2:.6e}")
                print(f"  Final combined Linf error: {final_linf:.6e}")

            if max_refinements == 0:
                print(
                    f"[Info] Reference solver {results[reference_index].name} uses the base resolution (no refinements requested)."
                )
            elif convergence_reached:
                print(
                    f"[Info] Reference solver {results[reference_index].name} converged at level {reference_refinement_level} "
                    f"with aggregate L2 difference {last_difference:.3e} (tolerance {convergence_tol:.3e})."
                )
            else:
                print(
                    f"[Warn] Reference solver {results[reference_index].name} did not reach the tolerance within {max_refinements} refinements; "
                    f"using level {reference_refinement_level} with final aggregate L2 difference {last_difference:.3e}."
                )

        return {
            "results": results,
            "reference_result": reference_result,
            "common_vars": common_vars,
            "combined_l2": combined_l2,
            "combined_linf": combined_linf,
            "per_var_l2": per_var_l2,
            "per_var_linf": per_var_linf,
            "grid_ratios": grid_ratios,
            "plots": {
                "Phi L2": phi_l2_plot,
                "Phi Linf": phi_linf_plot,
                "Pi L2": pi_l2_plot,
                "Pi Linf": pi_linf_plot,
            },
            "log": log_buffer.getvalue(),
        }
    finally:
        tempdir.cleanup()


## Configure and run solver comparison

Select the solvers to compare, adjust their parfiles, and run the workflow. The notebook writes temporary parfiles with your edits and reuses the plotting utilities from `compare_solvers.py`.


In [4]:
solver_configs = discover_solvers()
config_order = [cfg.key for cfg in solver_configs]
config_map = {cfg.key: cfg for cfg in solver_configs}

solver_selector = widgets.SelectMultiple(
    options=[(cfg.name, cfg.key) for cfg in solver_configs],
    value=tuple(config_order[:2]),
    description="Solvers",
    rows=min(6, len(solver_configs)),
    layout=widgets.Layout(width="100%"),
)

parfile_editors: Dict[str, widgets.Textarea] = {}
reset_buttons: Dict[str, widgets.Button] = {}
parfile_panels: Dict[str, widgets.VBox] = {}

for cfg in solver_configs:
    editor = widgets.Textarea(
        value=cfg.read_parfile(),
        layout=widgets.Layout(width="100%", height="220px"),
    )
    parfile_editors[cfg.key] = editor

    reset_btn = widgets.Button(description="Reset to default", button_style="")

    def _make_reset(key: str):
        def _reset(_):
            editor_text = config_map[key].read_parfile()
            parfile_editors[key].value = editor_text
        return _reset

    reset_btn.on_click(_make_reset(cfg.key))
    reset_buttons[cfg.key] = reset_btn

    panel = widgets.VBox(
        [
            widgets.HTML(f"<b>{cfg.name}</b>"),
            editor,
            reset_btn,
        ]
    )
    parfile_panels[cfg.key] = panel

reference_selector = widgets.Dropdown(description="Reference")
refine_factor_input = widgets.BoundedIntText(value=2, min=1, description="Refine")
max_refinements_input = widgets.BoundedIntText(value=4, min=0, description="Levels")
convergence_tol_input = widgets.FloatText(value=1e-4, description="L2 tol")
run_button = widgets.Button(description="Run comparison", button_style="success", icon="play")

status_output = widgets.Output(layout=widgets.Layout(border="1px solid #bbb", max_height="260px", overflow_y="auto"))
plots_output = widgets.Output()

parfile_container = widgets.VBox()
parfile_container.layout = widgets.Layout(width="65%")

solver_controls = widgets.VBox(
    [
        solver_selector,
        widgets.HBox([reference_selector, refine_factor_input]),
        widgets.HBox([max_refinements_input, convergence_tol_input]),
        run_button,
    ]
)
solver_controls.layout = widgets.Layout(width="35%")

controls_layout = widgets.HBox([solver_controls, parfile_container])


def _update_reference_options(*_):
    ordered = ordered_subset(solver_selector.value, config_order)
    if not ordered:
        reference_selector.options = []
        reference_selector.value = None
        return
    reference_selector.options = [(config_map[key].name, key) for key in ordered]
    if reference_selector.value not in ordered:
        reference_selector.value = ordered[0]


def _update_parfile_panels(*_):
    ordered = ordered_subset(solver_selector.value, config_order)
    parfile_container.children = tuple(parfile_panels[key] for key in ordered)


def _run_clicked(_):
    ordered = ordered_subset(solver_selector.value, config_order)
    if len(ordered) < 2:
        with status_output:
            clear_output()
            print("Select at least two solvers before running.")
        plots_output.clear_output()
        return
    if reference_selector.value not in ordered:
        with status_output:
            clear_output()
            print("Reference solver must be one of the selected solvers.")
        plots_output.clear_output()
        return

    for key in ordered:
        if not parfile_editors[key].value.strip():
            with status_output:
                clear_output()
                print(f"Parfile for {config_map[key].name} is empty; please provide parameters.")
            plots_output.clear_output()
            return

    ref_idx = ordered.index(reference_selector.value)

    solver_inputs = [
        (config_map[key], parfile_editors[key].value)
        for key in ordered
    ]

    try:
        data = run_comparison(
            solver_inputs,
            reference_index=ref_idx,
            refine_factor=int(refine_factor_input.value),
            convergence_tol=float(convergence_tol_input.value),
            max_refinements=int(max_refinements_input.value),
        )
    except Exception as exc:
        with status_output:
            clear_output()
            print(f"Run failed: {exc}")
        plots_output.clear_output()
        return

    with status_output:
        clear_output()
        print(data["log"])

    plots_output.clear_output()
    with plots_output:
        summary_lines = ["**Solver summaries**"]
        for result in data["results"]:
            final_l2 = data["combined_l2"][result.name][-1]
            final_linf = data["combined_linf"][result.name][-1]
            summary_lines.append(
                f"- Solver {result.name} (`{Path(result.solver_path).name}`) → "
                f"L2={final_l2:.3e}, Linf={final_linf:.3e}, grid ratio {data['grid_ratios'][result.name]}×"
            )
        summary_lines.append(
            f"- Reference comparison used solver {data['results'][ref_idx].name} "
            f"with {len(data['common_vars'])} shared variable(s)."
        )
        display(Markdown("".join(summary_lines)))

        available_plots = {label: path for label, path in data["plots"].items() if path is not None}
        if available_plots:
            display(Markdown("**Generated error plots**"))
            for label, path in available_plots.items():
                display(Markdown(label))
                display(Image(filename=str(path)))
        else:
            display(Markdown("_No error plots were generated._"))


solver_selector.observe(_update_reference_options, names="value")
solver_selector.observe(_update_parfile_panels, names="value")
run_button.on_click(_run_clicked)

_update_reference_options()
_update_parfile_panels()

display(controls_layout)
display(Markdown("### Run output"))
display(status_output)
display(plots_output)


### Run output

Output(layout=Layout(border_bottom='1px solid #bbb', border_left='1px solid #bbb', border_right='1px solid #bb…

Output()

# XProf

In [6]:
import os, sysconfig, shutil
scripts = sysconfig.get_path("scripts")
os.environ["PATH"] = scripts + os.pathsep + os.environ["PATH"]
print("which tensorboard (after):", shutil.which("tensorboard"))

%load_ext tensorboard
%tensorboard --logdir=profiles/ --host=127.0.0.1 --port=0

which tensorboard (after): /home/isuds/Gravity/Rationals/venv/bin/tensorboard
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
